## Baixar bibliotecas



In [1]:
!!pip install pillow

['Requirement already satisfied: pillow in c:\\users\\lucas\\appdata\\local\\programs\\python\\python310\\lib\\site-packages (9.2.0)']

## Importando as bibliotecas

In [8]:
from PIL import Image


## Abrindo a imagem e a convertendo para escala de cinza

In [34]:
im = Image.open('img.jpg').convert('L')
im2 = Image.open('img2.jpg').convert('L')
im.show()
im2.show()

## Usando o algoritimo de treshold

In [35]:
threshold = 190
im = im.point(lambda p: p > threshold and 255) #Mapear a imagem no metodo threshold
im2 = im2.point(lambda p: p > threshold and 255) #Mapear a imagem no metodo threshold

## Criando a nova imagem que só contem os pixel da frente 

In [36]:
im = im.convert('RGBA')
new_im = Image.new('RGBA', im.size, (255, 255, 255))
new_im.paste(im, mask=im)

im2 = im2.convert('RGBA')
new_im2 = Image.new('RGBA', im2.size, (255, 255, 255))
new_im2.paste(im2, mask=im2)



## Salvando a imagem

In [39]:
new_im.save('image_bg_removed.png')
nova = Image.open("image_bg_removed.png")
nova.show()

In [40]:
new_im2.save('image_bg_removed2.png')
nova2 = Image.open("image_bg_removed2.png")
nova2.show()

In [41]:


import numpy as np
import matplotlib.pyplot as plt
import cv2
from numpy.lib.type_check import imag

minimum = 4000                  #Define a área de contorno mínimo
frame1 = None
cap = cv2.VideoCapture(0)   # Ativa a camera como captura
method = 'ABS'

# meetodo de subtracao de fundo
mog = cv2.createBackgroundSubtractorMOG2()  
knn = cv2.createBackgroundSubtractorKNN()   

while True:
    ret, frame = cap.read()
    vid = cv2.flip(frame,1)
    
    if method == 'MOG2':
        bgs = mog.apply(vid)
        
    
    elif method == 'KNN':
        bgs = knn.apply(vid)

    
    elif method == 'ABS':
        frame = cv2.GaussianBlur(vid,(7,7),0)
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        
        if frame1 is None:
            frame1 = frame
            continue 
        
        framedelta = cv2.absdiff(frame1,frame)
        retval, bgs = cv2.threshold(framedelta.copy(), 50, 255, cv2.THRESH_BINARY)
    
    mask = np.zeros_like(frame)

# Encontrando contornos e os imprimindo na tela
    
    contours,_ = cv2.findContours(bgs, mode= cv2.RETR_TREE, method= cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours,key=cv2.contourArea,reverse= True)
    
    for cnt in contours:
        if cv2.contourArea(cnt) < minimum:
                continue
        
        (x,y,w,h) = cv2.boundingRect(cnt)
        cv2.rectangle(vid,(x,y),(x+w,y+h),(0,255,10),1)
        cv2.putText(vid,f'{method}',(20,20),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,0,2))
        cv2.putText(vid,'Motion Detected',(20,40),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,0,2))
        cv2.drawContours(mask,cnt,-1,255,3)
        break

    cv2.imshow('frame',vid)
    cv2.imshow('BGS',bgs)


    key = cv2.waitKey(1)
    if key == ord('q') or key == ord('Q'):
        break
    elif key == ord('M') or key == ord('m'):
        method = 'MOG2'
    elif key == ord('K') or key == ord('k'):
        method = 'KNN'
    elif key == ord('A') or key == ord('a'):
        method = 'ABS'
            
cap.release()
cv2.destroyAllWindows()